In [1]:
import numpy as np
import pandas as pd

In [2]:
# import standard PyTorch modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

import tqdm
import time

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [104]:
from datasets import load_dataset

dataset = load_dataset("nell", "nell_belief")

Reusing dataset nell (/Users/lemon/.cache/huggingface/datasets/nell/nell_belief/1115.0.0/0b3db04630c4a5967fc48a0539dd09ed64510d63837975cd187cf73d99f6e3ff)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['entity', 'relation', 'value', 'iteration_of_promotion', 'score', 'source', 'entity_literal_strings', 'value_literal_strings', 'best_entity_literal_string', 'best_value_literal_string', 'categories_for_entity', 'categories_for_value', 'candidate_source'],
        num_rows: 2766079
    })
})

In [105]:
entity = dataset['train']['entity'][1 : 10001]

6335

In [106]:
relation = dataset['train']['relation'][1 : 10001]

In [103]:

for j in range(300):
    print(entity[j])
    print(relation[j])

concept:biotechcompany:aspect_medical_systems
generalizations
concept:company:limited_brands
concept:companyceo
concept:company:limited_brands
generalizations
concept:company:limited_brands
generalizations
concept:biotechcompany:calavo_growers
generalizations
concept:biotechcompany:playboy_enterprises_inc
generalizations
concept:biotechcompany:b_c_s_b_bancorp_inc
generalizations
concept:biotechcompany:boeing_integrated_defense_systems
generalizations
concept:company:mahindra___mahindra_ltd_
generalizations
concept:biotechcompany:marsam_pharmaceuticals
generalizations
concept:biotechcompany:sifco_industries_inc
generalizations
concept:biotechcompany:medecins_sans_frontieres
concept:organizationalsoknownas
concept:biotechcompany:acceleron
concept:haswikipediaurl
concept:biotechcompany:acceleron
generalizations
concept:biotechcompany:midwestone_financial_group_inc
generalizations
concept:company:verifone
generalizations
concept:biotechcompany:anntaylor_stores_corp
generalizations
concept:

In [78]:
len(set(relation))

61

In [107]:
concept = dataset['train']['value'][1 : 10001]

In [108]:
best_entity_ls = dataset['train']["best_entity_literal_string"][1 : 10001]

In [109]:
best_concept_ls = dataset['train']["best_value_literal_string"][1 : 10001]

In [113]:
df = pd.DataFrame( {"entity" : entity, "concept" : concept, "relation" : relation} )

In [111]:
df.to_csv("nell.csv", index = False)

In [69]:
df = pd.DataFrame( {"entity" : entity, "concept" : concept, "relation" : en_relation} )
df.to_csv("nell-new.csv", index = False)

In [5]:
df_node = pd.read_csv("nell-node.csv", sep = ",")

In [6]:
df_node

,entity,relation,best-entity-query,best-concept-query,query
0,concept:biotechcompany:aspect_medical_systems,generalizations,Aspect Medical Systems,NaN,Aspect Medical Systems
1,concept:company:limited_brands,concept:companyceo,limited brands,Leslie-Wexner,limited brands Leslie-Wexner
2,concept:company:limited_brands,generalizations,limited brands,NaN,limited brands
3,concept:company:limited_brands,generalizations,limited brands,NaN,limited brands
4,concept:biotechcompany:calavo_growers,generalizations,Calavo Growers,NaN,Calavo Growers
...,...,...,...,...,...
9643,concept:biotechcompany:the_immune_response_corp,generalizations,the_immune_response_corp,NaN,the_immune_response_corp
9644,concept:biotechcompany:tb_woods_corp,generalizations,tb woods corp,NaN,tb woods corp
9645,concept:biotechcompany:lan_airlines_s_a,generalizations,Lan Airlines S.A,NaN,Lan Airlines S.A
9646,concept:biotechcompany:orbital_sciences_corp,generalizations,orbital sciences corp,NaN,orbital sciences corp


In [2]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub

module_url = "./universal/"

tf.__version__

'1.15.0'

In [4]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Message: Elephant
Embedding size: 512
Embedding: [0.04498472437262535, -0.05743393674492836, 0.002211477607488632, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.055680178105831146, -0.00960793811827898, 0.006246280390769243, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.03874939680099487, 0.0765201672911644, -0.0007945839897729456, ...]



In [10]:
query = df_node['query']

equery = []

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
    query_embeddings = session.run(embed(query))

    for i, embedding in enumerate(np.array(query_embeddings).tolist()):
        equery.append(embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [41]:
feature = pd.DataFrame( {"entity" : df_node['entity'], 'embedded-query' : equery} )
feature.to_csv("nell-node-feature.csv", sep = '\t', index = False)

In [44]:
new_feature = pd.read_csv("nell-node-feature.csv", sep = '\t')
new_feature.to_csv("nell-node-feature.gz", sep = '\t', index = False, compression = 'gzip')

In [15]:
data = pd.read_csv("nell-new.csv", sep = ',', header = None)
len(data[0].unique()), len(data[1].unique()), len(data[2].unique())

(6182, 1730, 60)